# Wildfire Risk - Predicting with XGBoost Model
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [3]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

In [25]:
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26
#!pip list

import boto3
import sagemaker
import pandas as pd
import json
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)


In [23]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-30-03-35-20'
auto_ml_job_name_wf                                   -> 'automl-fire-03-05-42-40'
auto_ml_job_name_wildfire                             -> 'automl-fire-06-16-52-30'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:857283526476:endpoint
autopilot_endpoint_arn_wf                             -> 'arn:aws:sagemaker:us-east-1:857283526476:endpoint
autopilot_endpoint_wf                                 -> 'automl-dm-ep-30-04-47-49'
autopilot_endpoint_wildfire                           -> 'automl-wildfire-ep-12-04-55-39'
autopilot_model_arn_wf                                -> 'arn:aws:sagemaker:us-east-1:857283526476:model/au
autopilot_model_wf                                    -> 'automl-dm-model-30-04-47-47'
autopilot_test_s3_wildfire                            -> 's3://sagemaker-us-east-1-857283526476/data/wildfi
autopilot_train_s3_

In [28]:
%store -r autopilot_endpoint_wildfire
%store -r autopilot_test_s3_wildfire

In [29]:
try:
    autopilot_endpoint_wildfire
    print("[OK]")
except NameError:
    print("***************************************************************************")
    print("[ERROR] PLEASE WAIT FOR THE PREVIOUS NOTEBOOK TO FINISH *******************")
    print("[ERROR] OR THIS NOTEBOOK WILL NOT RUN PROPERLY ****************************")
    print("***************************************************************************")

[OK]


In [30]:
print(autopilot_endpoint_wildfire)

automl-wildfire-ep-12-04-55-39


In [31]:
sm.get_waiter("endpoint_in_service").wait(EndpointName=autopilot_endpoint_wildfire)

In [32]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_wildfire)
status = resp["EndpointStatus"]

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Arn: arn:aws:sagemaker:us-east-1:857283526476:endpoint/automl-wildfire-ep-12-04-55-39
Status: InService


# Predict Test Data with XGBoost Model

In [38]:
#See the prepared testingdata
!aws s3 cp $autopilot_test_s3_wildfire ./tmp/

test_df = pd.read_csv("./tmp/wildfire_risk_autopilot_test.csv")
test_df.head()

download: s3://sagemaker-us-east-1-857283526476/data/wildfire_risk_autopilot_test.csv to tmp/wildfire_risk_autopilot_test.csv


,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,CDSD,CDSD_ATTRIBUTES,CLDD,CLDD_ATTRIBUTES,DT00,...,CARBON_DOWN_DEAD,CARBON_LITTER,CARBON_SOIL_ORG,CARBON_STANDING_DEAD,CARBON_UNDERSTORY_AG,CARBON_UNDERSTORY_BG,STATE,WATERCD,LAT,LON
0,USR0000CLAU,40.1417,-120.3533,1767.8,"LAUFMAN CALIFORNIA, CA US",383.8,U,151.6,",U",0.0,...,0.359501,14.140916,12.305077,0.575000,1.786990,0.198554,CA,4.0,40.123705,-120.265761
1,USR0000COZE,34.6819,-119.3539,1118.6,"OZENA CALIFORNIA, CA US",0.0,U,0.0,",U",0.0,...,0.274181,9.412408,11.751685,0.000000,1.778610,0.197623,CA,0.0,34.717939,-119.337134
2,USR0000CSIM,41.0750,-122.3733,731.5,"SIMS CALIFORNIA, CA US",677.4,U,199.5,",U",0.0,...,2.221914,15.884487,22.216451,2.940016,1.326997,0.147444,CA,0.0,41.179919,-122.434149
3,USR0000OLAV,43.9300,-121.3300,1418.8,"LAVA BUTTE OREGON, OR US",39.5,NaN,29.2,",U",0.0,...,0.414716,7.717282,34.189353,5.734431,1.732395,0.192488,OR,0.0,43.919335,-121.261092
4,USR0000OSLI,44.4625,-120.2944,1737.4,"SLIDE MOUNTAIN OREGON, OR US",0.3,U,0.3,",U",0.0,...,7.157813,18.814765,27.720882,8.171223,0.893344,0.099260,OR,0.0,44.347085,-120.362049


In [44]:
predictor = Predictor(model_name= autopilot_endpoint_wildfire, 
                      endpoint_name = autopilot_endpoint_wildfire, 
                      sagemaker_session = sess, 
                      serializer=CSVSerializer(), 
                      desearlizer=JSONDeserializer())

response = predictor.predict(test_df.to_csv(index=False).encode('utf-8'))
predictions = response.decode('utf-8')
predictions_df = pd.read_csv(io.StringIO(predictions), header=None)




ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from container-1 with message "Received data of unknown size. Expected number of columns is 1. Number of columns in the received data is 343701.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/automl-wildfire-ep-12-04-55-39 in account 857283526476 for more information.

In [40]:
predictions_df

KeyError: 0

# Specify the S3 Location of the Features

In [ ]:
%store -r processed_train_data_s3_uri

In [ ]:
try:
    processed_train_data_s3_uri
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("+++++++++++++++++++++++++++++++")

In [ ]:
print(processed_train_data_s3_uri)

In [ ]:
%store -r processed_validation_data_s3_uri

In [ ]:
try:
    processed_validation_data_s3_uri
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous sections before you continue.")
    print("+++++++++++++++++++++++++++++++")

In [ ]:
print(processed_validation_data_s3_uri)

In [ ]:
%store -r processed_test_data_s3_uri

In [ ]:
try:
    processed_test_data_s3_uri
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous sections before you continue.")
    print("+++++++++++++++++++++++++++++++")

In [ ]:
print(processed_test_data_s3_uri)

In [ ]:
print(processed_train_data_s3_uri)
!aws s3 ls $processed_train_data_s3_uri/

In [ ]:
print(processed_validation_data_s3_uri)
!aws s3 ls $processed_validation_data_s3_uri/

In [ ]:
print(processed_test_data_s3_uri)
!aws s3 ls $processed_test_data_s3_uri/

In [ ]:
!pip install sagemaker-experiments
!pip list

In [ ]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri, distribution="ShardedByS3Key")
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri, distribution="ShardedByS3Key")
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri, distribution="ShardedByS3Key")

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

In [ ]:
!cat src/tf_bert_reviews.py